In [154]:
import pandas as pd
from datetime import datetime
import requests
import numpy as np
from polygon.rest import RESTClient
from tqdm import tqdm
import warnings
import pytz
from concurrent.futures import ThreadPoolExecutor, as_completed
import concurrent.futures
import json
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

api_key = 'hFrBS7nzcaLTa8mplO1ejm44DI4EscDM'
client = RESTClient(api_key)

warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.simplefilter(action='ignore', category=pd_errors.SettingWithCopyWarning)
warnings.simplefilter("ignore", category=UserWarning)

In [155]:
df = pd.read_csv('dirty_main.csv')

<h1>Get market cap and ticker type</h1>

In [98]:
def unix(date_str,hour,minute):
    date = datetime.strptime(date_str, '%Y-%m-%d').date()
    # Define Eastern Standard Time (EST) timezone
    est_tz = pytz.timezone('America/New_York')
    # Define the time for 2 PM EST
    est_time = datetime(date.year, date.month, date.day, hour, minute, 0)
    # Localize the datetime object to EST timezone
    est_dt = est_tz.localize(est_time, is_dst=None)
    # Convert the datetime object to UTC timezone
    utc_dt = est_dt.astimezone(pytz.utc)
    # Calculate the Unix timestamp in milliseconds
    unix_timestamp_ms = int((utc_dt - datetime(1970, 1, 1, tzinfo=pytz.utc)).total_seconds() * 1000)
    return unix_timestamp_ms

In [28]:
session = requests.Session()
retries = Retry(total=50, backoff_factor=1, status_forcelist=[500, 502, 503, 504, 429])
session.mount("https://", HTTPAdapter(max_retries=retries))

def market_cap(ticker, date):
    try:
        url = f'https://api.polygon.io/v3/reference/tickers/{ticker}?date={date}&apiKey={api_key}'
        response = session.get(url, timeout=10)  # Add timeout
        response.raise_for_status()  # Raise error for bad responses
        data_dict = response.json()
        if data_dict['status'] == 'OK':
            if 'market_cap' in data_dict['results']:
                mktcap = data_dict['results']['market_cap']
            else:
                mktcap = np.nan
            if 'type' in data_dict['results']:
                t = data_dict['results']['type']
            else:
                t = np.nan
        else:
            mktcap = t = np.nan    
        return mktcap, t
    except Exception as e:
        print(f"An error occurred for {ticker} on {date}: market cap {e}")
        mktcap = np.nan
    return mktcap
market_cap('BIOA','2024-09-26')

An error occurred for BIOA on 2024-09-26: market cap 404 Client Error: Not Found for url: https://api.polygon.io/v3/reference/tickers/BIOA?date=2024-09-26&apiKey=hFrBS7nzcaLTa8mplO1ejm44DI4EscDM


nan

In [29]:
det_list = []
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    results = list(tqdm(executor.map(market_cap, df['ticker'], df['date']), total=len(df)))
df['market_cap'], df['type'] = zip(*results)
df

 13%|█▎        | 30150/232666 [03:37<49:33, 68.10it/s]  

An error occurred for BIOA on 2024-09-27: market cap 404 Client Error: Not Found for url: https://api.polygon.io/v3/reference/tickers/BIOA?date=2024-09-27&apiKey=hFrBS7nzcaLTa8mplO1ejm44DI4EscDM
An error occurred for BIOA on 2024-09-26: market cap 404 Client Error: Not Found for url: https://api.polygon.io/v3/reference/tickers/BIOA?date=2024-09-26&apiKey=hFrBS7nzcaLTa8mplO1ejm44DI4EscDM


100%|██████████| 232666/232666 [30:06<00:00, 128.80it/s] 


TypeError: 'float' object is not iterable

In [156]:
results_df = pd.DataFrame(results, columns=['market_cap', 'type'])
df = pd.concat([df, results_df], axis=1)

<h1>Get index prices % changes</h1>

In [124]:
def est_all(unix_ms_timestamp):
    # Convert Unix timestamp in milliseconds to seconds
    unix_seconds = unix_ms_timestamp / 1000.0
    # Create a datetime object from the Unix timestamp
    utc_time = datetime.utcfromtimestamp(unix_seconds)
    # Define the UTC and EST timezones
    utc_zone = pytz.utc
    est_zone = pytz.timezone('US/Eastern')
    # Localize the UTC datetime object to UTC timezone
    utc_time = utc_zone.localize(utc_time)
    # Convert the UTC time to EST
    est_time = utc_time.astimezone(est_zone)
    # Remove timezone info and return formatted date
    est_time = est_time.replace(tzinfo=None)
    return est_time.strftime('%Y-%m-%d')
est_all(1641805920000)

'2022-01-10'

In [157]:
def index_daily(ticker):
    data = client.get_aggs(ticker,
                            multiplier=1,
                            timespan='day',
                            adjusted='true',
                            from_='2020-12-31',
                            to='2024-09-30')
    df = pd.DataFrame(data)
    df['date'] = df['timestamp'].apply(est_all)
    df['c-1'] = df['close'].shift(1)
    df[f'{ticker}_overnight'] = (df['open'].shift(-1)/df['close'])-1
    df[f'{ticker}_dailyc'] = (df['close']/df['close'].shift(1))-1
    return df[['date','c-1',f'{ticker}_overnight',f'{ticker}_dailyc']]
index_daily('SPY')

,date,c-1,SPY_overnight,SPY_dailyc
0,2020-12-31,NaN,0.003825,NaN
1,2021-01-04,373.88,-0.001871,-0.013614
2,2021-01-05,368.79,-0.004363,0.006887
3,2021-01-06,371.33,0.006826,0.005979
4,2021-01-07,373.55,0.003930,0.014857
...,...,...,...,...
937,2024-09-24,569.67,-0.000280,0.002861
938,2024-09-25,571.30,0.007614,-0.002205
939,2024-09-26,570.04,0.001905,0.003965
940,2024-09-27,572.30,-0.001837,-0.001450


In [158]:
day_dict = {}
for ticker in ['SPY','VONE','IWM','VTHR','VTI']:
    day_dict[ticker] = index_daily(ticker)

In [159]:
def index_minute(ticker,date):
    data = client.get_aggs(ticker,
                            multiplier=1,
                            timespan='minute',
                            adjusted='true',
                            from_=unix(date,9,30),
                            to=unix(date,15,58),
                            limit=50000)
    df = pd.DataFrame(data)
    df['date'] = df['timestamp'].apply(est_all)
    return df['close'].iloc[-1]
index_minute('AAPL','2021-01-04')

129.27

In [160]:
for ticker in ['SPY','VONE','IWM','VTHR','VTI']:
    frame = day_dict[ticker]
    with concurrent.futures.ThreadPoolExecutor() as executor:
        list358 = list(tqdm(executor.map(index_minute, [ticker] * len(frame['date']),frame['date']), total=len(frame)))
    frame['358c'] = list358
    frame[f'{ticker}_dailya'] = (frame['358c']/frame['c-1'])-1

100%|██████████| 942/942 [00:15<00:00, 60.36it/s]


In [161]:
for ticker in ['SPY','VONE','IWM','VTHR','VTI']:
    frame = day_dict[ticker][[f'{ticker}_overnight',f'{ticker}_dailyc',f'{ticker}_dailya','date']]
    merged_df = pd.merge(df, frame, on='date', how='left')
    merged_df.rename(columns={f'{ticker}_overnight': f'{ticker}_ah',f'{ticker}_dailyc': f'{ticker}_pc_day',f'{ticker}_dailya': f'{ticker}_pc_timed'}, inplace=True)
    df = merged_df

In [162]:
df.to_csv('dirty_main2.csv')